# Exercise: More metrics derived from confusion matrices

* Please make sure unit 3 and 5 are done first so you can copy+paste critical code and we can be consistent in our message
* Please also read previous content in the word doc
* This is a short unit! You can be explicit about this in the introduction.
* Copy code used to build the model used in unit 5
* Build a confusion matrix (again, copy code)
* Manually calculate the following metrics. These will be introduced in the content. The goal here is to explain these in the context of your results:
* Accuracy
* Sensitivity/Recall
* Specificity
* Precision
* False positive rate

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

#Let's have a look at the data
dataset

TODO - everything else! Up to you, please try to keep it at a reasonable length by maximising code-reuse within this unit and from previous modules where possible.